In [29]:
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

import pycountry

from glob import glob

# Carregar dados

Dataset disponível em:
https://www.kaggle.com/datasets/unsdsn/world-happiness/data

In [30]:
files=glob('../dados/archive/*.csv')
files

['../dados/archive/2016.csv',
 '../dados/archive/2017.csv',
 '../dados/archive/2015.csv',
 '../dados/archive/2019.csv',
 '../dados/archive/2018.csv']

In [31]:
#Carregando os dados para um dicionário
reports={}
for path in files:
    ano=path.split('/')[-1].split('.')[0] #retorna o ano contido no caminho do arquivo
    reports[ano]=pd.read_csv(path)

# Descrição dos dados

O dataset contém reports sobre o nível de felicidade em 155 países entre os anos de 2015 e 2019. O nível de felicidade foi medido em uma pesquisa em que pessoas eram solicitadas a dar uma nota para suas pŕoprias vidas, na qual 10 seria o melhor cenário possível e 0 o pior. Os reports contam com o score de felicidade obtido por cada país, além de seis indicadores que podem ser usados para explicar o índice de felicidade. Os indicadores são os seguintes:

- GDP Per capita é o PIB per capita do pais;

- Social support é a média das repostas para a seguinte  pergunta: Em caso de dificuldades, você conta com familiares ou amigos que possam te ajudar? 

- Healthy life expectancy é a expectativa de vida;	

- Freedom to make life choices é a média das repostas para a seguinte pergunta: Você está safisfeito com a liberdade que possui para fazer escolhas na sua vida?;
 
- Generosity é o resíduo da regressão entre GDP per capita e a resposta média para a seguinte pergunta: Você doou dinheiro para caridade no mês passado?;

- Perceptions of corruption é a média das respostas para duas perguntas. A primeira delas é se a corrupção está disseminada no governo e a segunda se a corrupção está disseminada no mundo empresarial. Essa variável varia entre 0 e 1, onde 1 seria o país menor corrupto;

Para o cálculo da média das respostas foi atribuído valor 1 para respostas potivivas e valor 0 para respostas negativas.

Explicação detalhada pode ser encontrada em : https://worldhappiness.report/faq/

- Dystopia é um pai imaginário que teria o menor índice de felicidade. Esse país foi criado para servir com parâmetros de comparação com os demais, ou seja, como certo país está em comparação com o país na pior situação possível. Dystopia tem o pior resultado para cada um dos seis indicadores considerados;

- Residuals indicam a extensão com que os incicadores conseguem ou não explicar os indices de felicidade.

# Análise Preliminar

In [32]:
for ano in reports.keys():
    print(ano+f' : {reports[ano].shape}')
    print(reports[ano].columns)
    print('')

#O número de colunas e de linhas não é igual em todos os reports   

2016 : (157, 13)
Index(['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Lower Confidence Interval', 'Upper Confidence Interval',
       'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
       'Freedom', 'Trust (Government Corruption)', 'Generosity',
       'Dystopia Residual'],
      dtype='object')

2017 : (155, 12)
Index(['Country', 'Happiness.Rank', 'Happiness.Score', 'Whisker.high',
       'Whisker.low', 'Economy..GDP.per.Capita.', 'Family',
       'Health..Life.Expectancy.', 'Freedom', 'Generosity',
       'Trust..Government.Corruption.', 'Dystopia.Residual'],
      dtype='object')

2015 : (158, 12)
Index(['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Standard Error', 'Economy (GDP per Capita)', 'Family',
       'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)',
       'Generosity', 'Dystopia Residual'],
      dtype='object')

2019 : (156, 9)
Index(['Overall rank', 'Country or region', 'Score', 'GDP per ca

- Nos reports de 2015,2016 e 2017 o indicador social support foi chamado de family;
- Reports de 2015, 2016 e 2017 contam com a coluna  'Dystopia Residual';
- Reports de 2016 e 2017 contam com o intervalo de confiança do indicado de felicidade;
- Nos reports de 2015 a 2017 a percepção de corrupção está relacionada somente ao setor governamental (Trust (Government Corruption)), enquanto nos de 2018 e 2019 está relacionada tanto ao setor governamental quanto ao setor empresarial.


In [33]:
#Alterando o nome das colunas para que tenham nomes iguais em reports diferentes

reports['2019'].rename(columns={'Country or region':'Country'},inplace=True)
reports['2018'].rename(columns={'Country or region':'Country'},inplace=True)
reports['2016'].rename(columns={'Happiness Rank':'Overall rank','Happiness Score':'Score','Economy (GDP per Capita)':'GDP per capita',
                                'Family':'Social support','Health (Life Expectancy)':'Healthy life expectancy','Freedom':'Freedom to make life choices'},inplace=True)
reports['2017'].rename(columns={ 'Happiness.Rank':'Overall rank','Happiness.Score':'Score','Economy..GDP.per.Capita.':'GDP per capita',
                                'Family':'Social support','Health..Life.Expectancy.':'Healthy life expectancy',
                                'Freedom':'Freedom to make life choices','Trust..Government.Corruption.':'Trust (Government Corruption)'},inplace=True)
reports['2015'].rename(columns={'Happiness Rank':'Overall rank','Happiness.Score':'Score','Economy (GDP per Capita)':'GDP per capita',
                                'Family':'Social support','Health (Life Expectancy)':'Healthy life expectancy',
                                'Freedom':'Freedom to make life choices','Trust (Government Corruption)':'Trust (Government Corruption)'},inplace=True)

In [34]:
#Adicionando uma coluna com o ano em cada um dos reports
for ano in reports.keys():
    reports[ano]['Year']=int(ano)

In [35]:
#Concatenando os dados dos reports anuais para facilitar a análise da evolução anual dos indicadores

reports_2015_a_2019=pd.concat([reports[ano] for ano in reports.keys()],axis=0)

In [36]:
reports_2015_a_2019

,Country,Region,Overall rank,Score,Lower Confidence Interval,Upper Confidence Interval,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Trust (Government Corruption),Generosity,Dystopia Residual,Year,Whisker.high,Whisker.low,Dystopia.Residual,Happiness Score,Standard Error,Perceptions of corruption
0,Denmark,Western Europe,1,7.526,7.460,7.592,1.44178,1.16374,0.79504,0.57941,0.44453,0.36171,2.73939,2016,NaN,NaN,NaN,NaN,NaN,NaN
1,Switzerland,Western Europe,2,7.509,7.428,7.590,1.52733,1.14524,0.86303,0.58557,0.41203,0.28083,2.69463,2016,NaN,NaN,NaN,NaN,NaN,NaN
2,Iceland,Western Europe,3,7.501,7.333,7.669,1.42666,1.18326,0.86733,0.56624,0.14975,0.47678,2.83137,2016,NaN,NaN,NaN,NaN,NaN,NaN
3,Norway,Western Europe,4,7.498,7.421,7.575,1.57744,1.12690,0.79579,0.59609,0.35776,0.37895,2.66465,2016,NaN,NaN,NaN,NaN,NaN,NaN
4,Finland,Western Europe,5,7.413,7.351,7.475,1.40598,1.13464,0.81091,0.57104,0.41004,0.25492,2.82596,2016,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,Yemen,NaN,152,3.355,NaN,NaN,0.44200,1.07300,0.34300,0.24400,NaN,0.08300,NaN,2018,NaN,NaN,NaN,NaN,NaN,0.064
152,Tanzania,NaN,153,3.303,NaN,NaN,0.45500,0.99100,0.38100,0.48100,NaN,0.27000,NaN,2018,NaN,NaN,NaN,NaN,NaN,0.097
153,South Sudan,NaN,154,3.254,NaN,NaN,0.33700,0.60800,0.17700,0.11200,NaN,0.22400,NaN,2018,NaN,NaN,NaN,NaN,NaN,0.106
154,Central African Republic,NaN,155,3.083,NaN,NaN,0.02400,0.00000,0.01000,0.30500,NaN,0.21800,NaN,2018,NaN,NaN,NaN,NaN,NaN,0.038


In [37]:
#Selecionando as colunas de interesse
selected_columns=reports['2019'].columns.to_list()
selected_columns=selected_columns+['Trust (Government Corruption)']
reports_2015_a_2019=reports_2015_a_2019.loc[:,selected_columns]

In [38]:
reports_2015_a_2019

,Overall rank,Country,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Year,Trust (Government Corruption)
0,1,Denmark,7.526,1.44178,1.16374,0.79504,0.57941,0.36171,NaN,2016,0.44453
1,2,Switzerland,7.509,1.52733,1.14524,0.86303,0.58557,0.28083,NaN,2016,0.41203
2,3,Iceland,7.501,1.42666,1.18326,0.86733,0.56624,0.47678,NaN,2016,0.14975
3,4,Norway,7.498,1.57744,1.12690,0.79579,0.59609,0.37895,NaN,2016,0.35776
4,5,Finland,7.413,1.40598,1.13464,0.81091,0.57104,0.25492,NaN,2016,0.41004
...,...,...,...,...,...,...,...,...,...,...,...
151,152,Yemen,3.355,0.44200,1.07300,0.34300,0.24400,0.08300,0.064,2018,NaN
152,153,Tanzania,3.303,0.45500,0.99100,0.38100,0.48100,0.27000,0.097,2018,NaN
153,154,South Sudan,3.254,0.33700,0.60800,0.17700,0.11200,0.22400,0.106,2018,NaN
154,155,Central African Republic,3.083,0.02400,0.00000,0.01000,0.30500,0.21800,0.038,2018,NaN


In [39]:
#Organizando para que os os indicadores anuais para cada pais apareçam na sequência
reports_2015_a_2019.sort_values(by=['Country','Year'],inplace=True)
reports_2015_a_2019

,Overall rank,Country,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Year,Trust (Government Corruption)
152,153,Afghanistan,NaN,0.319820,0.302850,0.303350,0.234140,0.365100,NaN,2015,0.097190
153,154,Afghanistan,3.360,0.382270,0.110370,0.173440,0.164300,0.312680,NaN,2016,0.071120
140,141,Afghanistan,3.794,0.401477,0.581543,0.180747,0.106180,0.311871,NaN,2017,0.061158
144,145,Afghanistan,3.632,0.332000,0.537000,0.255000,0.085000,0.191000,0.036,2018,NaN
153,154,Afghanistan,3.203,0.350000,0.517000,0.361000,0.000000,0.158000,0.025,2019,NaN
...,...,...,...,...,...,...,...,...,...,...,...
114,115,Zimbabwe,NaN,0.271000,1.032760,0.334750,0.258610,0.189870,NaN,2015,0.080790
130,131,Zimbabwe,4.193,0.350410,0.714780,0.159500,0.254290,0.185030,NaN,2016,0.085820
137,138,Zimbabwe,3.875,0.375847,1.083096,0.196764,0.336384,0.189143,NaN,2017,0.095375
143,144,Zimbabwe,3.692,0.357000,1.094000,0.248000,0.406000,0.132000,0.099,2018,NaN


# Plotagem de Mapas

In [40]:
#Adicionando os códigos dos países nos reports para possibilitar a plotagem de mapas
#Os países serão identificados nos mapa pelos seus códigos

def gerar_codigo_pais(coluna):
    '''
    Função que gera o código do país a partir do seu nome. Esse códigos são necessários para a criação de mapas

    Inputs:
        Coluna de um pandas dataframe com nomes de países

    Outputs:
        Python list com os códigos dos países
    '''
    codigo=[]
    for nome_pais in coluna:
        try:
            codigo.append(pycountry.countries.get(name=nome_pais).alpha_3)
            #alpha_3 é para gerar um código de três letras
        except:
            codigo.append(None)
    return codigo
        

In [41]:
#Adicionando a coluna com o código do pais nos reports
for ano in reports.keys():
    reports[ano]['Country_Code']=gerar_codigo_pais(reports[ano]['Country'])


### Mapas com indicadores para ano selecionado

Geração dos mapas:

- Os mapas podem ser gerados para os reports anuais. Os anos disponíveis estão entre 2015 e 2019;
- Os mapas podem ser gerados para os seguintes indicadores:

    - Overall rank;
    - Healthy life expectancy	
    - Score;
    - Freedom to make life choices;
    - Generosity;
    - Perceptions of corruption;
    - GDP per capita;
    - Social support.


In [42]:
def gerar_mapa(data,ano,indicador,projecao='natural earth',animation_frame=None):
    '''
    Função para gerar o mapa mundi com cada país colorido de acordo com o valor do indicador selecionado

    ano       - (str) Ano do report entre 2015 e 2019. Refere-se ao ano dos indicadores.
    indicador - (str) Indicador que será mostrado no mapa. Indicadores aceitos:  overall rank, score, freedom to make life choices, generosity, perceptions of corruption , 
    GDP per capita, social support.
    '''
    if ano not in ['2015','2016','2017','2018','2019']:
        raise Exception('Report indisponivel. Os reports disponíveis são os dos anos 2015 a 2019')
    if indicador not in ['Overall rank','Score','Freedom to make life choices','Generosity','Perceptions of corruption','GDP per capita','Social support','Healthy life expectancy']:
        raise Exception('Indicador não conhecido')
    if indicador=='Perceptions of corruption' and ano not in ['2019','2018']:
        indicador='Trust (Government Corruption)'
    fig = px.choropleth(data[ano], locations='Country_Code', color=indicador, hover_name='Country',animation_frame=animation_frame,
                    projection=projecao, title=indicador+' '+ano)
    fig.show()


In [43]:
#indicadores:['Overall rank','Score','Freedom to make life choices','Generosity','Perceptions of corruption','GDP per capita','Social support']
gerar_mapa(reports,'2019','GDP per capita')  

#https://www.datacamp.com/tutorial/making-map-in-python-using-plotly-library-guide

### Mapas com evolução do indicador selecionado ao longo dos anos

In [44]:
#Adicão da coluna com os códigos dos países
reports_2015_a_2019['Country_Code']=gerar_codigo_pais(reports_2015_a_2019['Country'])
reports_2015_a_2019.head()

,Overall rank,Country,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Year,Trust (Government Corruption),Country_Code
152,153,Afghanistan,NaN,0.319820,0.302850,0.303350,0.23414,0.365100,NaN,2015,0.097190,AFG
153,154,Afghanistan,3.360,0.382270,0.110370,0.173440,0.16430,0.312680,NaN,2016,0.071120,AFG
140,141,Afghanistan,3.794,0.401477,0.581543,0.180747,0.10618,0.311871,NaN,2017,0.061158,AFG
144,145,Afghanistan,3.632,0.332000,0.537000,0.255000,0.08500,0.191000,0.036,2018,NaN,AFG
153,154,Afghanistan,3.203,0.350000,0.517000,0.361000,0.00000,0.158000,0.025,2019,NaN,AFG


In [45]:
def mapa_animado(data,indicador,projecao='natural earth',animation_frame='Year'):
    '''
    Função para gerar o mapa mundi com cada país colorido de acordo com o valor do indicador selecionado
    No mapa será mostrado a evolução do indicador a longo dos anos, de 2015 a 2019

    indicador - (str) Indicador que será mostrado no mapa. Indicadores aceitos:  Overall rank, Score, Freedom to make life choices, Generosity, Perceptions of corruption,
    Trust (Government Corruption), GDP per capita, Social support.
    '''

    if indicador not in ['Overall rank','Score','Freedom to make life choices','Generosity','Perceptions of corruption','Trust (Government Corruption)','GDP per capita',
                         'Social support','Healthy life expectancy']:
        raise Exception('Indicador não conhecido')
    
    fig = px.choropleth(data, locations='Country_Code', color=indicador, hover_name='Country',animation_frame=animation_frame,
                    projection=projecao,title=indicador)
    fig.show()


In [46]:
mapa_animado(reports_2015_a_2019,'Perceptions of corruption')

# Evolução anual dos Indicadores

In [47]:
reports_2015_a_2019.columns

Index(['Overall rank', 'Country', 'Score', 'GDP per capita', 'Social support',
       'Healthy life expectancy', 'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption', 'Year', 'Trust (Government Corruption)',
       'Country_Code'],
      dtype='object')

In [91]:
#Calcular desvio padrão dos indicadores
desvio_padrao_indicadores=reports_2015_a_2019.groupby('Country').std(numeric_only=True)
desvio_padrao_indicadores.drop(columns='Year',inplace=True)
desvio_padrao_indicadores

,Overall rank,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Trust (Government Corruption)
Country,,,,,,,,,
Afghanistan,6.024948,0.265605,0.034180,0.198912,0.080217,0.087728,0.088569,0.007778,0.018606
Albania,6.618157,0.054510,0.044070,0.142736,0.060464,0.037015,0.023505,0.003536,0.012147
Algeria,21.004761,0.535610,0.060214,0.138801,0.073448,0.092699,0.008646,0.014849,0.013898
Angola,2.160247,0.040992,0.064094,0.219048,0.105597,0.051014,0.020917,NaN,0.008043
Argentina,9.148770,0.256293,0.055362,0.171728,0.077131,0.056039,0.026014,0.002828,0.012556
...,...,...,...,...,...,...,...,...,...
Venezuela,37.016213,0.627612,0.077735,0.181969,0.079676,0.122741,0.009033,0.003536,0.023248
Vietnam,8.871302,0.050953,0.057527,0.267161,0.080901,0.029501,0.044800,0.004243,0.014058
Yemen,6.348228,0.176611,0.127507,0.284191,0.065852,0.075736,0.010020,0.009192,0.011997


In [54]:
desvio_padrao_indicadores.sort_values(by='Overall rank',ascending=False)

,Overall rank,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Trust (Government Corruption)
Country,,,,,,,,,
Venezuela,37.016213,0.627612,0.077735,0.181969,0.079676,0.122741,0.009033,0.003536,0.023248
Swaziland,24.041631,NaN,0.069961,0.053853,0.053500,0.004540,0.076785,NaN,NaN
Lesotho,22.261701,0.003464,0.062666,0.077483,0.068696,0.044976,0.028754,0.013435,0.004204
Ivory Coast,21.684096,0.465100,0.050977,0.128850,0.079915,0.049483,0.028682,0.009192,0.024582
Benin,21.440616,0.626277,0.054011,0.137216,0.081358,0.050355,0.018661,0.010607,0.009821
...,...,...,...,...,...,...,...,...,...
Oman,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Puerto Rico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Somaliland Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
df_indicador=desvio_padrao_indicadores[[indicador]].reset_index()
df_indicador

,Country,Overall rank
0,Afghanistan,6.024948
1,Albania,6.618157
2,Algeria,21.004761
3,Angola,2.160247
4,Argentina,9.148770
...,...,...
165,Venezuela,37.016213
166,Vietnam,8.871302
167,Yemen,6.348228
168,Zambia,20.037465


In [ ]:
def maiores_variacoes_indicadores(reports,indicador,num_paises):
    desvio_padrao_indicadores=reports.groupby('Country').std(numeric_only=True)
    desvio_padrao_indicadores.drop(columns='Year',inplace=True)
    df_indicador=desvio_padrao_indicadores[[indicador]].reset_index()
    lista_paises=desvio_padrao_indicadores.sort_values(by=indicador,ascending=False).head(num_paises).index.tolist()
    resultado=reports_2015_a_2019[reports_2015_a_2019["Country"].isin(lista_paises)]
    resultado=resultado.loc[:,['Country',indicador,"Year"]]
    #resultado=resultado.pivot_table(columns="Country",index='Year')
    return resultado,lista_paises

In [78]:
lista_paises=desvio_padrao_indicadores.sort_values(by='Overall rank',ascending=False).head(5).index.tolist()
lista_paises

['Venezuela', 'Swaziland', 'Lesotho', 'Ivory Coast', 'Benin']

In [79]:
indicadores=desvio_padrao_indicadores.sort_values(by='Overall rank',ascending=False).head(5)[[indicador]].reset_index()
indicadores

,Country,Overall rank
0,Venezuela,37.016213
1,Swaziland,24.041631
2,Lesotho,22.261701
3,Ivory Coast,21.684096
4,Benin,21.440616


In [85]:
resultado=reports_2015_a_2019[reports_2015_a_2019["Country"].isin(lista_paises)]
resultado=resultado.loc[:,['Country',indicador]]
resultado


,Country,Overall rank
154,Benin,155
152,Benin,153
142,Benin,143
135,Benin,136
101,Benin,102
150,Ivory Coast,151
138,Ivory Coast,139
127,Ivory Coast,128
106,Ivory Coast,107
98,Ivory Coast,99


In [86]:
z=pd.merge(resultado,indicadores,on='Country')

In [89]:
z.sort_values(by='Overall rank_y',ascending=False)

,Country,Overall rank_x,Overall rank_y
20,Venezuela,108,37.016213
19,Venezuela,102,37.016213
18,Venezuela,82,37.016213
17,Venezuela,44,37.016213
16,Venezuela,23,37.016213
15,Swaziland,135,24.041631
14,Swaziland,101,24.041631
11,Lesotho,139,22.261701
13,Lesotho,144,22.261701
12,Lesotho,141,22.261701


In [70]:
def maiores_variacoes_indicadores(reports,indicador,num_paises):
    desvio_padrao_indicadores=reports.groupby('Country').std(numeric_only=True)
    desvio_padrao_indicadores.drop(columns='Year',inplace=True)
    #df_indicador
    lista_paises=desvio_padrao_indicadores.sort_values(by=indicador,ascending=False).head(num_paises).index.tolist()
    resultado=reports_2015_a_2019[reports_2015_a_2019["Country"].isin(lista_paises)]
    resultado=resultado.loc[:,['Country',indicador,"Year"]]
    #resultado=resultado.pivot_table(columns="Country",index='Year')
    return resultado,lista_paises

In [56]:
indicador='Overall rank'
numero_paises=10

In [71]:
x,y=maiores_variacoes_indicadores(reports_2015_a_2019,indicador,5)

In [72]:
y

['Venezuela', 'Swaziland', 'Lesotho', 'Ivory Coast', 'Benin']

In [69]:
]

,Country,Overall rank,Year
154,Benin,155,2015
152,Benin,153,2016
142,Benin,143,2017
135,Benin,136,2018
101,Benin,102,2019
150,Ivory Coast,151,2015
138,Ivory Coast,139,2016
127,Ivory Coast,128,2017
106,Ivory Coast,107,2018
98,Ivory Coast,99,2019


## Ranking de Felicidade

### Países cujas posições no ranking de felicidade mais sofreram variações

In [25]:
desvio_padrao_indicadores.sort_values(by='Overall rank',ascending=False).head()


,Overall rank,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Trust (Government Corruption)
Country,,,,,,,,,
Venezuela,37.016213,0.627612,0.077735,0.181969,0.079676,0.122741,0.009033,0.003536,0.023248
Swaziland,24.041631,NaN,0.069961,0.053853,0.053500,0.004540,0.076785,NaN,NaN
Lesotho,22.261701,0.003464,0.062666,0.077483,0.068696,0.044976,0.028754,0.013435,0.004204
Ivory Coast,21.684096,0.465100,0.050977,0.128850,0.079915,0.049483,0.028682,0.009192,0.024582
Benin,21.440616,0.626277,0.054011,0.137216,0.081358,0.050355,0.018661,0.010607,0.009821


In [26]:
maiores_dp_ranking=desvio_padrao_indicadores.sort_values(by='Overall rank',ascending=False).head().index.tolist()
maiores_dp_ranking

['Venezuela', 'Swaziland', 'Lesotho', 'Ivory Coast', 'Benin']

In [27]:
resultado=reports_2015_a_2019[reports_2015_a_2019['Country'].isin(maiores_dp_ranking)]
resultado=resultado.loc[:,['Overall rank','Country','Year']].reset_index(drop=True)
resultado

,Overall rank,Country,Year
0,155,Benin,2015
1,153,Benin,2016
2,143,Benin,2017
3,136,Benin,2018
4,102,Benin,2019
5,151,Ivory Coast,2015
6,139,Ivory Coast,2016
7,128,Ivory Coast,2017
8,107,Ivory Coast,2018
9,99,Ivory Coast,2019


In [28]:
df=resultado.pivot_table(columns=['Country'],index="Year")
df

Overall rank                                        
Country        Benin Ivory Coast Lesotho Swaziland Venezuela
Year                                                        
2015           155.0       151.0    97.0     101.0      23.0
2016           153.0       139.0     NaN       NaN      44.0
2017           143.0       128.0   139.0       NaN      82.0
2018           136.0       107.0   141.0       NaN     102.0
2019           102.0        99.0   144.0     135.0     108.0